# Malign

`malign` is a set of experimental methods for sequence alignment that allow:
    
- a unique alphabet for each sequence
- computation of k-best alignments
- usage of assymetric information
- perform single-pass true multiple alignment (instead of combination of pairwise)
- while usable with any type of sequence, including biological, it is intended for linguistic usage mostly
- gaps are full symbols, not "bad"
- incomplete scoring matrices

This notebook explores ideas on how to use it.

In [1]:
import itertools

# Import the library
import malign
from malign import tabulate_alms

The most stupid alignment method implemented in `malign` is the "dumb" one. It returns a single alignment for a number of sequences, padding spaces to the left and right without even considering the values of the sequences. It is used for bootstrapping a number of other alignment methods.

In [2]:
# Perform pairwise dumb alignment
seq_a = 'tra'
seq_b = 'fatata'
print(tabulate_alms(malign.multi_align([seq_a, seq_b], method="dumb")))

print()

# Perform multiwise dumb alignment
seqs = ['tra', 'fra', 'batata', 'virp', 'x']
print(tabulate_alms(malign.multi_align(seqs, method="dumb")))

| Idx   | Seq   |   Score |  #0  |  #1  |  #2  |  #3  |  #4  |  #5  |
|-------|-------|---------|------|------|------|------|------|------|
| 0     | A     |    -0.9 |  -   |  t   |  r   |  a   |  -   |  -   |
| 0     | B     |    -0.9 |  f   |  a   |  t   |  a   |  t   |  a   |

| Idx   | Seq   |   Score |  #0  |  #1  |  #2  |  #3  |  #4  |  #5  |
|-------|-------|---------|------|------|------|------|------|------|
| 0     | A     |    0.33 |  -   |  t   |  r   |  a   |  -   |  -   |
| 0     | B     |    0.33 |  -   |  f   |  r   |  a   |  -   |  -   |
| 0     | C     |    0.33 |  b   |  a   |  t   |  a   |  t   |  a   |
| 0     | D     |    0.33 |  -   |  v   |  i   |  r   |  p   |  -   |
| 0     | E     |    0.33 |  -   |  -   |  x   |  -   |  -   |  -   |


New advanced methods begin with our modified Needleman-Wunsch. The methods work on multidimensional transition matrices that include all symbols and gaps. All correspondences must be provided, even when dealing with the same alphabet.

For illustration, let's start with a common DNA base matrix for pairwise sequence alignment:

  |   | A  |  G |  C |  T | -  |
  |---|----|----|----|----|----|
  | A | 10 | -1 | -3 | -4 | -5 |
  | G | -1 | 7  | -5 | -3 | -5 |
  | C | -3 | -5 | 9  |  0 | -5 |
  | T | -4 | -3 |  0 |  8 |-5  |
  | - | -5 | -5 | -5 | -5 | 0  |

Note that in this case the matrix is symmetric, but this is not necessary as we'll see later. This matrix is available as `malign.utils.DNA_MATRIX`.

Here we start by performing some pairwise alignments on very simple genetic sequences. Note that the parameter `k` allows to return at most k alignments, in terms of best scores, if those are computed. This method is different from normal WN algorithm because it first computes alignments in terms of `seq_a` to `seq_b`, then in terms of `seq_b` to `seq_a`, and takes the mean score as representatitve of the alignment. The results are equal in this case, as the matrix is symmetric.

Note that this is actually running a multiwise alignment under the hood already.

In [3]:
# Perform pairwise, modifier NW alignment
print("===== Alignment 1")
seq_a = "GATTACA"
seq_b = "A"
print(tabulate_alms(malign.multi_align([seq_a, seq_b], k=2, method="anw", matrix=malign.utils.DNA_MATRIX)))

print()

print("===== Alignment 2")
seq_a = "GATTACA"
seq_b = "ATTT"
print(tabulate_alms(malign.multi_align([seq_a, seq_b], k=2, method="anw", matrix=malign.utils.DNA_MATRIX)))

===== Alignment 1
| Idx   | Seq   |   Score |  #0  |  #1  |  #2  |  #3  |  #4  |  #5  |  #6  |
|-------|-------|---------|------|------|------|------|------|------|------|
| 0     | A     |   -3.86 |  G   |  A   |  T   |  T   |  A   |  C   |  A   |
| 0     | B     |   -3.86 |  -   |  -   |  -   |  -   |  -   |  -   |  A   |

===== Alignment 2
| Idx   | Seq   |   Score |  #0  |  #1  |  #2  |  #3  |  #4  |  #5  |  #6  |
|-------|-------|---------|------|------|------|------|------|------|------|
| 0     | A     |       1 |  G   |  A   |  T   |  T   |  A   |  C   |  A   |
| 0     | B     |       1 |  -   |  A   |  T   |  T   |  -   |  T   |  -   |


But what if the matrix is not symmetric? This is not usually the case for genetic data, but let's for sake of experience simulate a condition where sequence B never has a T aligned with a C (from the "point of view" of A, the normal affinities hold). We simulate this by changing the matrix for (C, T), which is now different (not symmmetric) from (T, C), and check the results.

In [4]:
# Perform pairwise, assymetric NW
matrix = malign.utils.DNA_MATRIX.copy()
matrix['C', 'T'] = -99
print(tabulate_alms(malign.multi_align([seq_a, seq_b], k=4, method="anw", matrix=matrix)))

| Idx   | Seq   |   Score |  #0  |  #1  |  #2  |  #3  |  #4  |  #5  |  #6  |
|-------|-------|---------|------|------|------|------|------|------|------|
| 0     | A     |    0.43 |  G   |  A   |  T   |  T   |  A   |  C   |  A   |
| 0     | B     |    0.43 |  -   |  A   |  T   |  T   |  T   |  -   |  -   |
|       |       |         |      |      |      |      |      |      |      |
| 1     | A     |    0.43 |  G   |  A   |  T   |  T   |  A   |  C   |  A   |
| 1     | B     |    0.43 |  -   |  A   |  T   |  T   |  -   |  -   |  T   |


You can see that the algorithm now does its best not to align a C in sequence a to a T in sequence B. This can go to extremes, as in the first example below, even though the inverse alignment still work as it commonly does.

In [5]:
print("===== Alignment 1")
print(tabulate_alms(malign.multi_align(["GATTACA", "GATTATA"], k=2, method="anw", matrix=matrix)))

print()

print("===== Alignment 2")
print(tabulate_alms(malign.multi_align(["GATTATA", "GATTACA"], k=2, method="anw", matrix=matrix)))

===== Alignment 1
| Idx   | Seq   |   Score |  #0  |  #1  |  #2  |  #3  |  #4  |  #5  |  #6  |  #7  |
|-------|-------|---------|------|------|------|------|------|------|------|------|
| 0     | A     |    4.88 |  G   |  A   |  T   |  T   |  A   |  C   |  -   |  A   |
| 0     | B     |    4.88 |  G   |  A   |  T   |  T   |  A   |  -   |  T   |  A   |
|       |       |         |      |      |      |      |      |      |      |      |
| 1     | A     |    4.88 |  G   |  A   |  T   |  T   |  A   |  -   |  C   |  A   |
| 1     | B     |    4.88 |  G   |  A   |  T   |  T   |  A   |  T   |  -   |  A   |

===== Alignment 2
| Idx   | Seq   |   Score |  #0  |  #1  |  #2  |  #3  |  #4  |  #5  |  #6  |
|-------|-------|---------|------|------|------|------|------|------|------|
| 0     | A     |    7.57 |  G   |  A   |  T   |  T   |  A   |  T   |  A   |
| 0     | B     |    7.57 |  G   |  A   |  T   |  T   |  A   |  C   |  A   |


The assymetry starts to make more sense when we start dealing with different alphabets, or at least with sequences that indeed from different domains. Take the example below, manually crafted, where we prepare a matrix to align the orthography of Italian and Russian cognates, both written with the standard orthographies. Note how the matrix is assymetric even in terms of gaps: there are some cases where a gap is more or less likely in the correspondence -- sometimes they are even higher than the correspondence to other graphemes.

  |   | а  |  т |  о |  м | Я  | к  | в  | -  |
  |---|----|----|----|----|----|----|----|----|
  | a | 10 | -4 |  3 | -3 | 5  | -4 | -4 | -1 |
  | t | -4 | 10 | -4 | -3 | -4 | -1 | -3 | -3 |
  | o |  4 | -4 | 10 | -5 | 2  | -4 | -2 | 0  |
  | m | -2 | -1 | -4 |  9 |-3  | -2 | 3  | 1  |
  | G | -3 | 1  | -3 | -1 | -4 | 3  | -2 | 2  |
  | i | 5  | -3 | 4  | 0  | 6  | -3 | -3 | 2  |
  | c | -4 | -1 | -5 | -4 | -5 | 4  | 1  | -3 |
  | - | 1  | -2 | -1 | 2  | 2  | -4 | 2  | 0  |

In [6]:
ita_rus = malign.ScoringMatrix(filename="ita_rus.matrix")

print("===== Alignment 1")
print(tabulate_alms(malign.multi_align(["atomo", "атом"], k=2, method="anw", matrix=ita_rus)))

print()

print("===== Alignment 2")
print(tabulate_alms(malign.multi_align(["Giacomo", "Яков"], k=4, method="anw", matrix=ita_rus)))

===== Alignment 1
| Idx   | Seq   |   Score |  #0  |  #1  |  #2  |  #3  |  #4  |
|-------|-------|---------|------|------|------|------|------|
| 0     | A     |     7.4 |  a   |  t   |  o   |  m   |  o   |
| 0     | B     |     7.4 |  а   |  т   |  о   |  м   |  -   |

===== Alignment 2
| Idx   | Seq   |   Score |  #0  |  #1  |  #2  |  #3  |  #4  |  #5  |  #6  |  #7  |
|-------|-------|---------|------|------|------|------|------|------|------|------|
| 0     | A     |    2.86 |  G   |  i   |  a   |  c   |  o   |  m   |  o   |      |
| 0     | B     |    2.86 |  -   |  Я   |  -   |  к   |  о   |  в   |  -   |      |
|       |       |         |      |      |      |      |      |      |      |      |
| 1     | A     |    2.29 |  G   |  i   |  a   |  c   |  o   |  m   |  o   |      |
| 1     | B     |    2.29 |  -   |  Я   |  -   |  к   |  о   |  -   |  в   |      |
|       |       |         |      |      |      |      |      |      |      |      |
| 2     | A     |    2.12 |  G   |  i  

We can combine more domains for a multiple NW alignment; unlike most multiple NW methods, we don't build a tree internally because an evolutionary perspective does not necessarily apply in our cases (even in face of a unrooted tree); details are provided in the paper. Here we extend our matrix a bit with Greek cognates. Note that, for showcasing, we are only setting the actual correspondences, guiding the alignment a bit too much, and letting the algorithm fill the missing parts. Note that we add correspondences to two graphemes in Italian, "v", "s" which were not used in the example above.

Note that we start with sparse matrix without much information -- one Italian letter, `G`, has no single information.

The more balenced the information is in terms of strong and low association, the better it will usually be.

  |   | -   |  Ι |  α |  β | κ  | μ   | ο  | ς  | τ  | ω  |
  |---|-----|----|----|----|----|-----|----|----|----|----|
  | - |     | -3 |    |    | -7 | -10 |    | -3 |    |    |
  | G |     |    |    |    |    |     |    |    |    |    |
  | a | -7  | 2  | 10 |    |    |     | 6  |    |    | 4  |
  | c |     |    |    |    |  8 |     |    |    | 2  |    |
  | i | -3  | 7  | 5  |    |    |     | 3  |    |    |    |
  | m |     |    |    | 4  |    | 10  |    |    |    |    |
  | o | -7  |    | 4  |    |    |     | 10 |    |    | 10 |
  | s |     |    |    |    |    |     |    |  6 |    |    |
  | t | -10 |    |    |    |  2 |     |    |    | 10 |    |
  | v |     |    |    | 5  |    |     |    |    |    |    |

In [7]:
ita_grk = malign.ScoringMatrix(filename="ita_grk.matrix")


print("===== Alignment 1")
print(tabulate_alms(malign.multi_align(["atomo", "ατομο"], k=2, method="anw", matrix=ita_grk)))

print()

print("===== Alignment 2")
print(tabulate_alms(malign.multi_align(["Giacomo", "Ιακωβος"], k=4, method="anw", matrix=ita_grk)))

print()

# Combine the two matrices into a single one, add some points, show a couple of examples
full_matrix = malign.ScoringMatrix(scores={}, sub_matrices={(0,1):ita_rus, (0,2):ita_grk})
full_matrix['o', 'в', 'ο'] = -4
full_matrix['o', 'в', 'ς'] = -10
full_matrix['-', 'в', 'ς'] = -7.5
full_matrix['-', 'в', 'ο'] = -6.5
full_matrix['o', '-', 'ο'] = 5
full_matrix['o', '-', 'ς'] = -3
full_matrix['i', '-', 'Ι'] = -4
full_matrix['c', 'к', 'κ'] = 10
full_matrix['-', 'в', 'ς'] = -10
full_matrix['m', 'в', 'β'] = 10
for key in [('-', 'к', 'ο'), ('i', 'а', 'Ι'), ('m', 'в', 'β'), ('m', '-', 'β'),
            ('-', 'в', 'ς'), ('-', '-', 'ς'), ('o', '-', 'ς'), ('-', '-', 'ο'),
            ('-', 'в', 'ο')]:
    print(key, full_matrix[key])

# Do maligns
print("===== Alignment 1")
seqs = ['atomo', "атом", "ατομο"]
print(tabulate_alms(malign.multi_align(seqs, method="anw", k=4, matrix=full_matrix)))

print()

print("===== Alignment 2")
seqs = ["Giacomo", "Яков", "Ιακωβος"]
print(tabulate_alms(malign.multi_align(seqs, method="anw", k=4, matrix=full_matrix)))

===== Alignment 1
| Idx   | Seq   |   Score |  #0  |  #1  |  #2  |  #3  |  #4  |
|-------|-------|---------|------|------|------|------|------|
| 0     | A     |      10 |  a   |  t   |  o   |  m   |  o   |
| 0     | B     |      10 |  α   |  τ   |  ο   |  μ   |  ο   |

===== Alignment 2
| Idx   | Seq   |   Score |  #0  |  #1  |  #2  |  #3  |  #4  |  #5  |  #6  |  #7  |
|-------|-------|---------|------|------|------|------|------|------|------|------|
| 0     | A     |    4.58 |  G   |  i   |  a   |  c   |  o   |  m   |  o   |  -   |
| 0     | B     |    4.58 |  -   |  Ι   |  α   |  κ   |  ω   |  β   |  ο   |  ς   |

('-', 'к', 'ο') -1.5666666666666667
('i', 'а', 'Ι') 6.0
('m', 'в', 'β') 10
('m', '-', 'β') 2.5
('-', 'в', 'ς') -10
('-', '-', 'ς') -1.5
('o', '-', 'ς') -3
('-', '-', 'ο') 0.43333333333333335
('-', 'в', 'ο') -6.5
===== Alignment 1
| Idx   | Seq   |   Score |  #0  |  #1  |  #2  |  #3  |  #4  |  #5  |
|-------|-------|---------|------|------|------|------|------|------|
| 0 

We can experiment with the canonical example from List (2014)

In [8]:
seqs = ["VOLDEMORT", "WALDEMAR", "VLADIMIR", "VOLODYMIR"]

voldemort_matrix = malign.utils.identity_matrix(seqs)

In [9]:
for key in [('T', '-', '-', '-'),
            ('T', 'R', 'R', 'R'), ('-', 'R', 'R', 'R'), ('R', 'R', 'R', 'R'), ('O', 'A', 'I', 'I')]:
    print(key, voldemort_matrix[key])

print(tabulate_alms(malign.multi_align(seqs, method="anw", k=4, matrix=voldemort_matrix)))

('T', '-', '-', '-') -1.0
('T', 'R', 'R', 'R') 9.0
('-', 'R', 'R', 'R') 9.0
('R', 'R', 'R', 'R') 16.0
('O', 'A', 'I', 'I') 4.0
| Idx   | Seq   |   Score |  #0  |  #1  |  #2  |  #3  |  #4  |  #5  |  #6  |  #7  |  #8  |  #9  |
|-------|-------|---------|------|------|------|------|------|------|------|------|------|------|
| 0     | A     |    7.30 |  V   |  O   |  L   |  -   |  D   |  E   |  M   |  O   |  R   |  T   |
| 0     | B     |    7.30 |  W   |  A   |  L   |  -   |  D   |  E   |  M   |  A   |  R   |  -   |
| 0     | C     |    7.30 |  V   |  -   |  L   |  A   |  D   |  I   |  M   |  I   |  R   |  -   |
| 0     | D     |    7.30 |  V   |  O   |  L   |  O   |  D   |  Y   |  M   |  I   |  R   |  -   |
|       |       |         |      |      |      |      |      |      |      |      |      |      |
| 1     | A     |    6.44 |  V   |  O   |  L   |  D   |  E   |  M   |  O   |  R   |  T   |      |
| 1     | B     |    6.44 |  W   |  A   |  L   |  -   |  D   |  E   |  M   |  A   |  R   

The NW algorithm, or better, the implementation in `malign`, cannot guarantee a minimum number of k-best paths, and is subject to the same kind of issues of normal NW algorithms, such as differences in global and local alignments. One new experimental method offered by the library, that also allows true multisequence alignment, uses graph theory and Yen's algorithm in order to look for alignments. The same matrices are used for both methods, with all the common caveats, such as aligning gaps to states.

In [10]:
# Do maligns
print("===== Alignment 1")
seqs = ['atomo', "атом", "ατομο"]
print(tabulate_alms(malign.multi_align(seqs, method="yenksp", k=2, matrix=full_matrix)))
print()

print("===== Alignment 2")
seqs = ["Giacomo", "Яков", "Ιακωβος"]
print(tabulate_alms(malign.multi_align(seqs, method="yenksp", k=2, matrix=full_matrix)))
print()

print("===== Alignment 3")
seqs = ["VOLDEMORT", "WALDEMAR", "VLADIMIR", "VOLODYMIR"]
print(tabulate_alms(malign.multi_align(seqs, method="yenksp", k=4, matrix=voldemort_matrix)))
print()

for key in [('L', 'L', 'L', 'L')]:
    print(key, voldemort_matrix[key])

===== Alignment 1
| Idx   | Seq   |   Score |  #0  |  #1  |  #2  |  #3  |  #4  |
|-------|-------|---------|------|------|------|------|------|
| 0     | A     |    8.50 |  a   |  t   |  o   |  m   |  o   |
| 0     | B     |    8.50 |  а   |  т   |  о   |  м   |  -   |
| 0     | C     |    8.50 |  α   |  τ   |  ο   |  μ   |  ο   |
|       |       |         |      |      |      |      |      |
| 1     | A     |    7.20 |  a   |  t   |  o   |  m   |  o   |
| 1     | B     |    7.20 |  а   |  т   |  о   |  -   |  м   |
| 1     | C     |    7.20 |  α   |  τ   |  ο   |  μ   |  ο   |

===== Alignment 2
| Idx   | Seq   |   Score |  #0  |  #1  |  #2  |  #3  |  #4  |  #5  |  #6  |  #7  |
|-------|-------|---------|------|------|------|------|------|------|------|------|
| 0     | A     |    4.22 |  G   |  i   |  a   |  c   |  o   |  m   |  o   |  -   |
| 0     | B     |    4.22 |  -   |  Я   |  -   |  к   |  о   |  в   |  -   |  -   |
| 0     | C     |    4.22 |  -   |  Ι   |  α   |  κ   |  ω  

One immediate thing to notice is, as with any alignment method, the need for a scorer, which can be harder for our methods given the multidimensional, asymmetric, and non-square matrices. `malign` offers a number of facilities to compute and refine matrices, including some unsupervised learning that tries to maximize the alignment scores (and which generates a preliminary matrix that can be refined by the user).

We have been using them so far, let's investigate in more detail. Matrices will always try to fill missing point by learning from the ones provided. There are two main ways to provide seed data: either as a dictionary with full alignment sites, or as a submatrices (which can include full alignment sites). Let's start with the simplest, providing a full alignment site. We can initialize the matrix and check how both the data that was provided and that which was not are there (the latter being automatically filled in)

In [11]:
s = {
        ("a", "A", "1"): -1,
        ("a", "A", "2"): 4,
        ("a", "A", "3"): 3,
        ("a", "B", "1"): 1,
        ("b", "A", "1"): -10,
        ("b", "A", "2"): 10,
        ("b", "A", "3"): 10,
        ("b", "A", "4"): 10,
        ("c", "B", "1"): 2,
        ("c", "B", "2"): 2,
        ("a", "-", "-"): -2,
        ("b", "-", "-"): -2,
        ("-", "A", "-"): -20,
        ("-", "B", "-"): -20,
        ("-", "-", "1"): -3,
        ("a", "B", "-"): -10,
        ("-", "A", "1"): -100,
        ("-", "A", "2"): -10,
        ("-", "B", "3"): -5,
    }
m = malign.ScoringMatrix(s)
print(m["a", "A", "2"]) # provided
print(m["-", "-", "3"]) # inferred in first round
print(m["c", "-", "4"]) # inferred in second round

4
-4.0
-1.4394910644910646


We can now query submatrices, that in this case were inferred as well. In fact, as the product of possible domains is extremely large but usually we are only going to use `n` (for the number of domains), these are inferred on-demand and cached for reuse if necessary (using parameters for inferring that are set when instantiating the matrix).

In [12]:
print(m["a", "A", None])
print(m["a", None, "3"])
print(m[None, "B", "1"])

4
3
2


The alternative, as we already did, is to initialize the scoring matrix with submatrices, which can include corretion points for full domain. We do this by creating full matrices for the sub-domains and providing them when instantiating. Note how the Itlaian Greek matrix here as far less information and we are not providing a Russian/Greek scorer (even though we could).

In [13]:
ita_rus = {("a", "а"):10,
("a", "т"):-4,
("a", "о"):3,
("a", "м"):-3,
("a", "Я"):5,
("a", "к"):-4,
("a", "в"):-4,
("a", "-"):-1,

("t", "а"):-4,
("t", "т"):10,
("t", "о"):-4,
("t", "м"):-3,
("t", "Я"):-4,
("t", "к"):-1,
("t", "в"):-3,
("t", "-"):-3,

("o", "а"):4,
("o", "т"):-4,
("o", "о"):10,
("o", "м"):-5,
("o", "Я"):2,
("o", "к"):-4,
("o", "в"):-2,
("o", "-"):0,

("m", "а"):-2,
("m", "т"):-1,
("m", "о"):-4,
("m", "м"):9,
("m", "Я"):-3,
("m", "к"):-2,
("m", "в"):2,
("m", "-"):1,

("G", "а"):-3,
("G", "т"):1,
("G", "о"):-3,
("G", "м"):-1,
("G", "Я"):4,
("G", "к"):3,
("G", "в"):-2,
("G", "-"):2,

("i", "а"):5,
("i", "т"):-3,
("i", "о"):4,
("i", "м"):0,
("i", "Я"):6,
("i", "к"):-3,
("i", "в"):-3,
("i", "-"):2,

("c", "а"):-4,
("c", "т"):-1,
("c", "о"):-5,
("c", "м"):-4,
("c", "Я"):-5,
("c", "к"):4,
("c", "в"):1,
("c", "-"):-3,

("-", "а"):1,
("-", "т"):-2,
("-", "о"):-1,
("-", "м"):2,
("-", "Я"):2,
("-", "к"):-4,
("-", "в"):2,
("-", "-"):0,}

ita_grk = {    
  ("a", "α"):10,
  ("a", "-"):-5,
  ("-", "α"):-5,
  ("o", "α"):4,
  ("i", "α"):5,
  ("t", "τ"):10,
  ("c", "τ"):2,
  ("a", "ο"):6,
  ("o", "ο"):10,
  ("o", "-"):-10,
  ("-", "ο"):10,
  ("m", "μ"):10,
  ("a", "Ι"):2,
  ("i", "Ι"):7,
  ("t", "κ"):2,
  ("c", "κ"):8,
  ("a", "ω"):4,
  ("o", "ω"):10,
  ("m", "β"):4,
  ("v", "β"):5,
  ("s", "ς"):6,
}
    

ita_rus_m = malign.ScoringMatrix(ita_rus)
ita_grk_m = malign.ScoringMatrix(ita_grk)

print(ita_rus_m["c", "Я"]) # provided
print(ita_grk_m["t", "μ"]) # inferred

# we provide a single point ita/rus/grk
scorer = {("t", "т", "τ"):10}
sub_matrices = {
    (0, 1): ita_rus_m,
    (0, 2): ita_grk_m,
}
irg_m = malign.ScoringMatrix(scorer, sub_matrices=sub_matrices)

print(irg_m["c", "Я", None]) # provided
print(irg_m["t", None, "μ"]) # provided
print(irg_m[None, "Я", "μ"]) # inferred
print(irg_m["a", "а", "α"]) # inferred
print(irg_m["a", "а", "-"]) # inferred
print(irg_m["a", "Я", "μ"]) # inferred

irg_m.save("ita_rus_grk.matrix")

m2 = malign.ScoringMatrix(filename="ita_rus_grk.matrix")
print(m2["c", "Я", None]) # provided
print(m2["t", None, "μ"]) # provided
print(m2[None, "Я", "μ"]) # inferred
print(m2["a", "а", "α"]) # inferred
print(m2["a", "а", "-"]) # inferred
print(m2["a", "Я", "μ"]) # inferred

m2['a', "Я", None]

-5
7.333333333333333
-5.0
7.333333333333333
8.0
10.0
2.5
4.75
-5.0
7.333333333333333
8.0
10.0
2.5
4.75


5.0

## Scoring Matrices

Matrices in `malign` are ultimately just dictionaries that provide a score for alignment vectors, following an ordered list of domains (or alphabets), which are sets of symbols expressed by strings of at least one character (including the gap symbol, which for must purposes is a symbol like all others). Alignment sites can be full of partial, allowing to score sub-alignments; partial alignment vectors have the same length of the full vectors, with the unrelates sites marked by `None`. Partial alignment vectors must have at least two valid elements (and in most cases will, indeed, have only two, as used for pairwise alignments).

There is not much to decide on the scores themselves, as their organization is free. By definition, the alignment of full gap vectors (that is, vectors where all non-None sites are gaps) is zero, and there is a general tendency that positively correlated sites have positive numbers and negatively correlated sites have negative numbers, but this is not mandatory. In most applications gaps are usually aligned with lower scores, but this is not necessary and, in fact, it is one of the reasons `malign` was developed.

A scoring matrix might be complete or non-complete. Complete matrices are necessary for most operations and can generally be just Python dictionaries; for other cases, the `ScoringMatrix` object helps with the inference of missing data, allowing to complete a matrix.

Here we create one matrix of domain two and print it. By default, the first domain is printed on the vertical.

In [14]:
# Define a scoring matrix for a simple, two domain system
# alphabet a is {'-', 'a', 'b', 'c'}, alphabet b is {'-', 'X', 'Y'}
vectors = {
    ('-', '-') :  0.0,
    ('-', 'X') : -3.0,
    ('-', 'Y') : -9.0,
    ('a', '-') : -3.0,
    ('a', 'X') :  0.0,
    ('a', 'Y') :  8.0,
    ('b', '-') : -5.0,
    ('b', 'X') :  4.0,
    ('b', 'Y') :  4.0,
    ('c', '-') :  2.0,
    ('c', 'X') : -1.0,
    ('c', 'Y') :  7.0,
}

matrix = malign.ScoringMatrix(vectors)
print(matrix.tabulate())

|    |   - |   X |   Y |
|----|-----|-----|-----|
| -  |   0 |  -3 |  -9 |
| a  |  -3 |   0 |   8 |
| b  |  -5 |   4 |   4 |
| c  |   2 |  -1 |   7 |


If some vector is missing, the `ScoringMatrix` will infer it when initialized (for complete vectors) or when necessary with cache (for incomplete vectors). Different methods are offered for such inferece.

In [15]:
v = vectors.copy()
v.pop(('a', '-'))
v.pop(('c', 'Y'))
matrix = malign.ScoringMatrix(v)
print(matrix.tabulate())

|    |     - |   X |    Y |
|----|-------|-----|------|
| -  |  0    |  -3 | -9   |
| a  |  1.25 |   0 |  8   |
| b  | -5    |   4 |  4   |
| c  |  2    |  -1 |  0.8 |


As seen above, we can have partial vectors, and a matrix can be initialized with them as well.

We can use different filling method.

In [16]:
# Define a scoring matrix for a simple, three domain system
# alphabet a is {'-', 'a', 'b', 'c'}, alphabet b is {'-', 'X', 'Y'}, alphabet c is {'-', 'i', 'j'}
# note that we already comment out some entries to make gaps
vectors = {
    ('-', '-', '-') :  0.0,
#    ('-', '-', 'i') : -4.0,
    ("-", "-", "j") : -8.0,

    ('-', 'X', '-') : -5.0,
    ('-', 'Y', '-') : -5.0,
    ('a', '-', '-') :  0.0,
#    ('a', 'X', '-') :  0.0,
    ('a', 'Y', '-') :  8.0,
    ('b', '-', '-') :  0.0,
    ('b', 'X', '-') :  4.0,
    ('b', 'Y', '-') :  4.0,
    ('c', '-', '-') :  0.0,
    ('c', 'X', '-') : -1.0,
    ('c', 'Y', '-') : -7.0,    
    
    ('-', 'X', 'i') : -3.0,
    ('-', 'Y', 'i') : -9.0,
    ('a', '-', 'i') : -3.0,
#    ('a', 'X', 'i') :  0.0,
#    ('a', 'Y', 'i') :  8.0,
    ('b', '-', 'i') : -5.0,
    ('b', 'X', 'i') :  4.0,
    ('b', 'Y', 'i') :  4.0,
    ('c', '-', 'i') :  2.0,
    ('c', 'X', 'i') : -1.0,
    ('c', 'Y', 'i') :  7.0,
    
    ('-', 'X', 'j') : -5.0,
    ('-', 'Y', 'j') : -6.0,
    ('a', '-', 'j') :  3.0,
    ('a', 'X', 'j') :  8.0,
    ('a', 'Y', 'j') :  8.0,
    ('b', '-', 'j') : -6.0,
    ('b', 'X', 'j') :  5.0,
    ('b', 'Y', 'j') :  4.0,
    ('c', '-', 'j') : -5.0,
#    ('c', 'X', 'j') :  6.0,
    ('c', 'Y', 'j') :  7.0,
}

# TODO: have no fill method

print("Standard fill method")
matrix = malign.ScoringMatrix(vectors)
print(matrix.tabulate())

print("Distance fill method")
matrix = malign.ScoringMatrix(vectors, fill_method="distance")
print(matrix.tabulate())

Standard fill method
|    |   /-/- |     /-/i |   /-/j |     /X/- |   /X/i |     /X/j |   /Y/- |   /Y/i |   /Y/j |
|----|--------|----------|--------|----------|--------|----------|--------|--------|--------|
| -  |      0 | -4.33333 |     -8 | -5       |     -3 | -5       |     -5 |   -9   |     -6 |
| a  |      0 | -3       |      3 |  2.33333 |      1 |  8       |      8 |    2.5 |      8 |
| b  |      0 | -5       |     -6 |  4       |      4 |  5       |      4 |    4   |      4 |
| c  |      0 |  2       |     -5 | -1       |     -1 |  1.14286 |     -7 |    7   |      7 |
Distance fill method
|    |   /-/- |     /-/i |   /-/j |     /X/- |     /X/i |      /X/j |   /Y/- |     /Y/i |   /Y/j |
|----|--------|----------|--------|----------|----------|-----------|--------|----------|--------|
| -  |      0 | -2.39286 |     -8 | -5       | -3       | -5        |     -5 | -9       |     -6 |
| a  |      0 | -3       |      3 |  1.07692 |  1.08333 |  8        |      8 |  1.34615 |      8 

`ScoringMatrix` also much facilitate the inferece of full alignments from collections of partial alignments, as might be available in most cases, such as when performing initial training. These can be combined with any number of actual full alignment. The partial alignments can also be sparse and it is possible to have complete missing data, as for the 12 domain below.

Note the difference between individual pairwise scorers and a full one.

In [17]:
vector_01 = {
    ('-', 'X') : -3.0,
    ('a', '-') : -3.0,
    ('a', 'X') :  0.0,
    ('a', 'Y') :  8.0,
    ('b', '-') : -5.0,
    ('b', 'Y') :  4.0,
    ('c', 'X') : -1.0,
    ('c', 'Y') :  7.0,
}
vector_02 = {
    ('a', '-') : -4.0,
    ('a', 'i') :  2.0,
    ('a', 'j') :  2.0,
    ('b', 'i') : -5.0,
    ('b', 'j') :  9.0,
    ('c', '-') : -7.0,
    ('c', 'j') :  4.0,
}

print("Matrix (0,1)")
matrix01 = malign.ScoringMatrix(vector_01)
print(matrix01.tabulate())

print("Matrix (0,2)")
matrix02 = malign.ScoringMatrix(vector_02)
print(matrix02.tabulate())

print("Matrix (0,1,2)")
matrix = malign.ScoringMatrix(scores={}, sub_matrices={(0,1):matrix01, (0,2):matrix02})
print(matrix.tabulate())

Matrix (0,1)
|    |    - |   X |   Y |
|----|------|-----|-----|
| -  |  0   |  -3 |   4 |
| a  | -3   |   0 |   8 |
| b  | -5   |  -1 |   4 |
| c  | -0.5 |  -1 |   7 |
Matrix (0,2)
|    |     - |    i |   j |
|----|-------|------|-----|
| -  |  0    | -1.5 |   5 |
| a  | -4    |  2   |   2 |
| b  | -1.75 | -5   |   9 |
| c  | -7    | -1.5 |   4 |
Matrix (0,1,2)
|    |   /-/- |   /-/i |   /-/j |   /X/- |   /X/i |   /X/j |   /Y/- |   /Y/i |   /Y/j |
|----|--------|--------|--------|--------|--------|--------|--------|--------|--------|
| -  |  0     |  -0.75 |   2.5  | -1.5   |  -2.25 |    1   |  2     |   1.25 |    4.5 |
| a  | -3.5   |  -0.5  |  -0.5  | -2     |   1    |    1   |  2     |   5    |    5   |
| b  | -3.375 |  -5    |   2    | -1.375 |  -3    |    4   |  1.125 |  -0.5  |    6.5 |
| c  | -3.75  |  -1    |   1.75 | -4     |  -1.25 |    1.5 |  0     |   2.75 |    5.5 |


We can also quickly compute identity matrices when all domains are the same. They are automatically computed from list of sequences/sets for all domains.

In [18]:
seqs = ["ab", "aab", "bbb"]

id_matrix = malign.utils.identity_matrix(seqs, match=2, gap_score=-3)
print(id_matrix.tabulate())

|    |   /-/- |   /-/a |   /-/b |   /a/- |   /a/a |   /a/b |   /b/- |   /b/a |   /b/b |
|----|--------|--------|--------|--------|--------|--------|--------|--------|--------|
| -  |      0 |     -3 |     -3 |     -3 |      8 |     -3 |     -3 |     -3 |      8 |
| a  |     -3 |      8 |      1 |      8 |     27 |      8 |      1 |      8 |      8 |
| b  |     -3 |      1 |      8 |      1 |      8 |      8 |      8 |      8 |     27 |
